# Test notebook to exercise the alert consumers

Important: change this to the correct import!

In [46]:
!/usr/bin/pip3 install lasair

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [47]:
import sys
import json
from lasair import LasairError, lasair_consumer

## Normal Consumer

In [48]:
lsst = False
if lsst:
    endpoint = "https://lasair-lsst.lsst.ac.uk/api"
    kafka_server = 'lasair-lsst-kafka_pub.lsst.ac.uk:9092'
    topic        = "lasair_5test_kafka"
else:
    endpoint     = "https://lasair-ztf.lsst.ac.uk/api"
    kafka_server = 'kafka.lsst.ac.uk:9092'
    topic        = "lasair_2Zooniverse"

In [49]:
group_id     = 'test121'
consumer = lasair_consumer(
    kafka_server,    # kafka.lsst.ac.uk:9092
    group_id,        # any string
    topic            # topic name for your filter
)

n = 0
while n < 10:
    msg = consumer.poll(timeout=20)
    if msg is None:
        break
    if msg.error():
        print(str(msg.error()))
        break
    jmsg = json.loads(msg.value())
    print(json.dumps(jmsg, indent=2))
    n += 1
print('No more messages available')

{
  "objectId": "ZTF24abccvkc",
  "ramean": 309.23826220454544,
  "decmean": 83.38991404545453,
  "mjdmin": 60545.27922450006,
  "mjdmax": 60571.30292819999,
  "magrmin": 16.9877,
  "rmag": 18.1081,
  "classification": "ORPHAN",
  "score": "Within 2arcsec of PS1 star",
  "UTC": "2024-09-18 08:17:00"
}
{
  "objectId": "ZTF24abdlaaq",
  "ramean": 314.2406892117647,
  "decmean": 67.0111353235294,
  "mjdmin": 60550.243414400145,
  "mjdmax": 60571.301400499884,
  "magrmin": 19.1361,
  "rmag": 19.1687,
  "classification": "SN",
  "score": "Not Near PS1 star",
  "UTC": "2024-09-18 08:17:00"
}
{
  "objectId": "ZTF24aawtoiv",
  "ramean": 346.36595046086956,
  "decmean": 26.63653376521739,
  "mjdmin": 60545.27552080015,
  "mjdmax": 60571.318101900164,
  "magrmin": 17.812,
  "rmag": 18.6718,
  "classification": "SN",
  "score": "Not Near PS1 star",
  "UTC": "2024-09-18 08:18:38"
}


%3|1726647592.559|FAIL|rdkafka#consumer-7| [thrd:lasair-lsst-kafka_pub:9092/bootstrap]: lasair-lsst-kafka_pub:9092/bootstrap: Failed to resolve 'lasair-lsst-kafka_pub:9092': nodename nor servname provided, or not known (after 4ms in state CONNECT, 18 identical error(s) suppressed)


No more messages available


## Rate Limited Consumer

In [50]:
from lasair import RL_consumer
group_id     = 'test120'
sqllite_filename = './%s.sql' % topic
rc = RL_consumer.lasair_RL_consumer(
    kafka_server,       # kafka.lsst.ac.uk:9092
    group_id,           # any string
    topic,              # topic name for your filter
    sqllite_filename,   # past history
    interval=1,         # how many days before release  
    verbose=False
)

In [51]:
while 1:
    msg = rc.poll()
    if msg is None:
        break;
    objectId = json.loads(msg.value())['objectId']
    print('---> object %s found at %s' % (objectId, RL_consumer.now_human()))

---> object ZTF24abdlaaq found at 2024-09-18 09:20:16.107870


%3|1726647622.587|FAIL|rdkafka#consumer-7| [thrd:lasair-lsst-kafka_pub:9092/bootstrap]: lasair-lsst-kafka_pub:9092/bootstrap: Failed to resolve 'lasair-lsst-kafka_pub:9092': nodename nor servname provided, or not known (after 4ms in state CONNECT, 18 identical error(s) suppressed)


In [52]:
rc.printall()

lasair_2Zooniverse
At 2024-09-18 09:20:26.131506
ZTF24abdlaaq accepted 0.000 days ago, rejected 0 times
ZTF24aaxseec accepted 0.005 days ago, rejected 1 times
ZTF24abcfigw accepted 0.005 days ago, rejected 1 times
ZTF24abcrkus accepted 0.005 days ago, rejected 3 times
ZTF24abdlsja accepted 0.005 days ago, rejected 1 times
ZTF24abhexdw accepted 0.005 days ago, rejected 1 times
ZTF24abdfuqr accepted 0.005 days ago, rejected 1 times
ZTF24abcixyi accepted 0.005 days ago, rejected 1 times
ZTF24abcrpjm accepted 0.005 days ago, rejected 1 times
ZTF23abidemb accepted 0.005 days ago, rejected 1 times
ZTF24abcwlht accepted 0.005 days ago, rejected 1 times
ZTF23abscfuu accepted 0.005 days ago, rejected 1 times
ZTF24abdkfqu accepted 0.005 days ago, rejected 1 times
ZTF24aawjwxx accepted 0.005 days ago, rejected 1 times
ZTF23aaroyff accepted 0.005 days ago, rejected 1 times
ZTF24aasjjkf accepted 0.005 days ago, rejected 1 times
ZTF23aaotwwn accepted 0.005 days ago, rejected 1 times
ZTF24abcrdha acc